# Data Cleaning

In this part we are cleaning the data and extract new features from existing one.

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
import re
import os
import ast

In [25]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [26]:
df = pd.read_csv("../data/data.csv", 
                          encoding='utf_8', 
                          dtype = 'unicode',
                          parse_dates = True,
                          infer_datetime_format = True,
                          low_memory=False)
df = df.drop("Unnamed: 0", axis = 1)


In [27]:
categ = pd.read_csv("../data/categories.txt")
categ.Cuisine = categ.Cuisine.str.rstrip()
categ = categ['Cuisine'].tolist()

In [28]:
df = df[df.categories.str.contains('|'.join(categ))]

In [29]:
def str_to_dict(x):
    if type(x) == float:
        return {}
    else:
        return ast.literal_eval(x)

In [30]:
df['hours'] = df['hours'].apply(str_to_dict)
df = pd.concat([df.drop(['hours'], axis=1), df['hours'].apply(pd.Series)], axis=1)

In [31]:
temp = df.set_index('business_id').categories.str.split(', ', expand=True).stack()
temp = pd.get_dummies(temp).groupby(level=0).sum()
temp = temp.loc[:, (temp.sum() >= 500)]

In [32]:
result = pd.merge(df, temp, left_on='business_id', right_index=True,
                  how='inner', sort=False)
result = result.drop(['categories'], axis = 1)

In [33]:
result['Music'] = result['Music'].apply(str_to_dict)
result = pd.concat([result.drop(['Music'], axis=1), result['Music'].apply(pd.Series)], axis=1)

In [34]:
def musicfunc (row):
    if row['dj'] == True or row['background_music'] == True or row['jukebox'] == True or row['live'] == True or row['video'] == True or row['karaoke'] == True:
        return True
    else:
        return False
result['Anymusic'] = result.apply(lambda row: musicfunc(row), axis = 1)    
   

In [35]:
result['Anymusic'].value_counts()

False    49689
True      1487
Name: Anymusic, dtype: int64

In [36]:
result = result.loc[:, (result.isnull().sum(axis=0) <= 47000)]

In [37]:
import datetime
result[['Opening_Mon','Closing_Mon']] = result['Monday'].str.split('-',expand=True)
result[['Opening_Tue','Closing_Tue']] = result['Tuesday'].str.split('-',expand=True)
result[['Opening_Wed','Closing_Wed']] = result['Wednesday'].str.split('-',expand=True)
result[['Opening_Thu','Closing_Thu']] = result['Thursday'].str.split('-',expand=True)
result[['Opening_Fri','Closing_Fri']] = result['Friday'].str.split('-',expand=True)
result[['Opening_Sat','Closing_Sat']] = result['Saturday'].str.split('-',expand=True)
result[['Opening_Sun','Closing_Sun']] = result['Sunday'].str.split('-',expand=True)


In [38]:
for i in [col for col in result if col.startswith('Opening') or col.startswith('Closing')]:
    result[i] = pd.to_datetime(result[i], format = '%H:%M')

In [39]:
def working_hours(name, clhrs, ophrs):
    result['{}'.format(name)] = np.nan
    for index,row in result.iterrows():
        value = pd.Timedelta(row['{}'.format(clhrs)] - row['{}'.format(ophrs)]).seconds / 3600
        #datetime.datetime.combine(datetime.date.min, row['Closing_Wed']) - datetime.datetime.combine(datetime.date.min,row['Opening_Wed'] )
        result.set_value(index,'{}'.format(name),value)

In [40]:
working_hours('WD_Mon', 'Closing_Mon', 'Opening_Mon')
working_hours('WD_Tue', 'Closing_Tue', 'Opening_Tue')
working_hours('WD_Wed', 'Closing_Wed', 'Opening_Wed')
working_hours('WD_Thu', 'Closing_Thu', 'Opening_Thu')
working_hours('WD_Fri', 'Closing_Fri', 'Opening_Fri')
working_hours('WD_Sat', 'Closing_Sat', 'Opening_Sat')
working_hours('WD_Sun', 'Closing_Sun', 'Opening_Sun')

C:\Games\Python\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [41]:
for i in [col for col in result if col.startswith('Opening') or col.startswith('Closing')]:
    result[i] = result[i].dt.time

In [42]:
result = result.drop(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], axis = 1)
result = result[result.columns.drop(list(result.filter(regex='Closing')))]

In [43]:
wifi = []
for i in result['WiFi']:
    x = re.findall(r"'(.*?)'", str(i))
    wifi.append(x)

In [46]:
Alc = []
for i in result['Alcohol']:
    x = re.findall(r"'(.*?)'", str(i))
    Alc.append(x)

In [48]:
noise = []
for i in result['NoiseLevel']:
    x = re.findall(r"'(.*?)'", str(i))
    noise.append(x)

In [52]:
attire = []
for i in result['RestaurantsAttire']:
    x = re.findall(r"'(.*?)'", str(i))
    attire.append(x)

In [44]:
result['WiFi'] = wifi
result['WiFi'] = result['WiFi'].str[0]

In [47]:
result['Alcohol'] = Alc
result['Alcohol'] = result['Alcohol'].str[0]

In [49]:
result['NoiseLevel'] = noise
result['NoiseLevel'] = result['NoiseLevel'].str[0]

In [53]:
result['RestaurantsAttire'] = attire
result['RestaurantsAttire'] = result['RestaurantsAttire'].str[0]

In [56]:
result['BusinessParking'] = result['BusinessParking'].apply(str_to_dict)
result = pd.concat([result.drop(['BusinessParking'], axis=1), result['BusinessParking'].apply(pd.Series)], axis=1)
result['GoodForMeal'] = result['GoodForMeal'].apply(str_to_dict)
result = pd.concat([result.drop(['GoodForMeal'], axis=1), result['GoodForMeal'].apply(pd.Series)], axis=1)

In [58]:
result['Ambience'] = result['Ambience'].apply(str_to_dict)
result = pd.concat([result.drop(['Ambience'], axis=1), result['Ambience'].apply(pd.Series)], axis=1)

In [60]:
result = result.loc[:, (result.isnull().sum(axis=0) <= 47000)]

In [63]:
result = result.rename(columns={'RestaurantsPriceRange2':'Price'})

In [64]:
result.head()

,address,business_id,city,is_open,latitude,longitude,name,postal_code,review_count,stars,state,Alcohol,BikeParking,BusinessAcceptsCreditCards,Caters,GoodForKids,HasTV,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,Price,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,WheelchairAccessible,WiFi,American (New),American (Traditional),Arts & Entertainment,Asian Fusion,Bakeries,Barbeque,Bars,Beer,Breakfast & Brunch,Buffets,Burgers,Cafes,Canadian (New),Caribbean,Caterers,Chicken Wings,Chinese,Cocktail Bars,Coffee & Tea,Comfort Food,Delis,Desserts,Diners,Ethnic Food,Event Planning & Services,Fast Food,Food,Food Delivery Services,French,Gastropubs,Gluten-Free,Greek,Grocery,Halal,Hot Dogs,Ice Cream & Frozen Yogurt,Indian,Italian,Japanese,Juice Bars & Smoothies,Korean,Latin American,Lounges,Mediterranean,Mexican,Middle Eastern,Nightlife,Pizza,Pubs,Restaurants,Salad,Sandwiches,Seafood,Soup,Specialty Food,Sports Bars,Steakhouses,Sushi Bars,Tex-Mex,Thai,Vegan,Vegetarian,Vietnamese,Wine & Spirits,Wine Bars,Anymusic,Opening_Mon,Opening_Tue,Opening_Wed,Opening_Thu,Opening_Fri,Opening_Sat,Opening_Sun,WD_Mon,WD_Tue,WD_Wed,WD_Thu,WD_Fri,WD_Sat,WD_Sun,garage,street,validated,lot,valet,dessert,latenight,lunch,dinner,brunch,breakfast,romantic,intimate,classy,hipster,divey,touristy,trendy,upscale,casual
0,30 Eglinton Avenue W,QXAEGFB4oINsVuTFxEYKFQ,Mississauga,1,43.6054989743,-79.652288909,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON,full_bar,False,NaN,True,True,False,loud,False,casual,False,True,2,True,True,True,NaN,no,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,False,09:00:00,09:00:00,09:00:00,09:00:00,09:00:00,09:00:00,09:00:00,15.0,15.0,15.0,15.0,16.0,16.0,15.0,False,False,False,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True
1,"10110 Johnston Rd, Ste 15",gnKjwL_1w79qoiV3IC_xQQ,Charlotte,1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC,beer_and_wine,True,True,False,True,True,average,False,casual,False,True,2,True,True,True,NaN,no,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,False,17:30:00,NaT,17:30:00,17:30:00,17:30:00,17:30:00,17:30:00,4.0,NaN,4.0,4.0,4.5,4.5,3.5,False,False,False,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True
8,2450 E Indian School Rd,1Dfx3zM-rW4n-31KeC8sJg,Phoenix,1,33.4951941,-112.0285876,Taco Bell,85016,18,3.0,AZ,none,NaN,True,NaN,True,False,NaN,False,casual,False,True,1,False,NaN,True,NaN,no,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,False,07:00:00,07:00:00,07:00:00,07:00:00,07:00:00,07:00:00,07:00:00,17.0,17.0,17.0,18.0,18.0,18.0,17.0,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False
9,"119 Landings Dr, Ste 101",5t3KVdMnFgAYmSl1wYLhmA,Mooresville,1,35.5274098057,-80.8680032061,The Kilted Buffalo Langtree,28117,9,3.5,NC,beer_and_wine,True,True,NaN,False,True,average,True,NaN,NaN,True,1,True,NaN,NaN,NaN,free,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,10:00:00,10:00:00,10:00:00,10:00:00,10:00:00,10:00:00,12:00:00,15.0,15.0,15.0,15.0,15.0,15.0,13.0,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False
10,5981 Andrews Rd,fweCYi8FmbJXHCqLnwuk8w,Mentor-on-the-Lake,1,41.70852,-81.359556,Marco's Pizza,44060,16,4.0,OH,none,True,True,NaN,True,NaN,NaN,False,casual,True,True,2,False,NaN,True,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,10:00:00,10:00:00,10:00:00,10:00:00,10:00:00,10:00:00,10:00:00,14.0,14.

## New features

Adding name length as feature

In [65]:
result['name_length']  = result['name'].str.len()

## Save the clean data

In [66]:
result.to_csv('../data/data_clean.csv')